In [1]:
import findspark, sys

# Locate the Spark installation (add pyspark to sys.path, see https://github.com/minrk/findspark#readme)
findspark.init()
print(f'Using Spark located in {findspark.find()}.')

from pyspark.sql import SparkSession

# Create or get the Spark session (singleton) and the underlying Spark context
spark = SparkSession.builder.getOrCreate()

Using Spark located in /usr/local/spark/.


In [2]:
sys.path.append('../../../scripts')
from pyspark.sql.types import BooleanType, DoubleType, LongType, StringType, StructField, StructType, ArrayType, IntegerType
from pyspark.sql.functions import udf, col, collect_list
from pyspark.sql import functions as F, Window
import utils

# Sixth and Seventh Hypothesis

In the following notebook the following hypotheses, already analyzed in local (using standard python), will be replicated on the Big Data source located in the HDFS:
 - **Are players loyal to a specific game notoriety level (no matter which one)?**
 - **Do players who mostly review famous games also enjoy niche ones?**

*P.S:* The two hypotheses are considered together in this notebook since some data are shared.

## 6) Are players loyal to a specific game notoriety level (no matter which one)?

### Data import, preprocessing & obtain useful metrics to perform the analysis
<br>

#### Import reviews dataset from HDFS

In [3]:
# Define a reasonable schema for the reviews dataset
reviews_schema = StructType([
    StructField('base_review_id', LongType(), True),
    StructField('steamid', LongType(), True), 
    StructField('appid', LongType(), True),
    StructField('voted_up', BooleanType(), True),
    StructField('votes_up', LongType(), True),
    StructField('votes_funny', LongType(), True),
    StructField('weighted_vote_score', DoubleType(), True),
    StructField('playtime_forever', LongType(), True),
    StructField('playtime_at_review', LongType(), True),
    StructField('num_games_owned', LongType(), True),
    StructField('num_reviews', LongType(), True),
    StructField('review', StringType(), True),
    StructField('unix_timestamp_created', LongType(), True),
    StructField('unix_timestamp_updated', LongType(), True)
])

# Read the reviews dataset from HDFS
base_reviews_df = spark.read.csv(
    path='hdfs://localhost:54310/final_project/data/base_reviews_filtered',
    schema=reviews_schema,
    escape='"',
    header=True,
    ignoreTrailingWhiteSpace=True,
    mode='FAILFAST',
    multiLine=True,
    unescapedQuoteHandling='STOP_AT_CLOSING_QUOTE'
).to_pandas_on_spark()

#### Create games dataframe - contains the number of written reviews for a specific game (overall & only positive)

In [4]:
# Obtain all reviews written for each game
games_df = base_reviews_df.appid.value_counts().reset_index().rename(columns={
    'index': 'appid',
    'appid': 'num_reviews'
})

# Obtain all positive reviews written for each game
pos_revs_per_game_df = base_reviews_df[base_reviews_df.voted_up].appid.value_counts().reset_index().rename(columns={
    'index': 'appid',
    'appid': 'pos_reviews'
})

# Merging the two dataframes
games_df = games_df.merge(pos_revs_per_game_df, how='left').fillna(0).astype(int)

#### Obtain for each game a notoriety level and a percentage of positive reviews

The notoriety level is basically a measure of how much a game has been discussed in the small reviews dataset.

In [5]:
s = games_df.num_reviews.quantile([.25, .85])
THRESH1 = s[.25]
THRESH2 = s[.85]

THRESH1, THRESH2

(14.0, 296.0)

In [6]:
def num_revs_to_label(num_rev: int) -> str:
    """ Get notoriety label from number of reviews """
    if num_rev < THRESH1:
        return 'NICHE'
    elif THRESH1 <= num_rev < THRESH2:
        return 'KNOWN'
    else:
        return 'FAMOUS'

udf_num_revs_to_label = udf(lambda x: num_revs_to_label(x), StringType())

games_df = games_df.to_spark().withColumn('notoriety', udf_num_revs_to_label(col('num_reviews'))).to_pandas_on_spark()
games_df['perc_pos_revs'] = (games_df.pos_reviews / games_df.num_reviews) * 100
games_df.to_spark().limit(100).toPandas()

,appid,num_reviews,pos_reviews,notoriety,perc_pos_revs
0,730,195377,171980,FAMOUS,88.024691
1,394360,10015,9171,FAMOUS,91.572641
2,232090,6788,5969,FAMOUS,87.934590
3,289070,9900,8002,FAMOUS,80.828283
4,433340,6259,6124,FAMOUS,97.843106
...,...,...,...,...,...
95,435150,10849,10379,FAMOUS,95.667803
96,48700,9247,9066,FAMOUS,98.042608
97,304390,5259,3385,FAMOUS,64.365849
98,594570,7550,7082,FAMOUS,93.801325


#### Calculate the maximum loyalty of each user (that have reviewed at least 3 games) to a game notoriety level

- **max_user_loyalty = (max(user_reviewed_games[notoriety_level]) / reviewed_games) * 100**

In [7]:
# Merging projected games and base reviews dataframe on appid
base_reviews_and_games = (base_reviews_df[['steamid', 'appid', 'voted_up']]
    .merge(games_df[['appid', 'notoriety']])
)

# Obtain for each user the number of written reviews he/she wrote for each level of notoriety
w1 = Window.partitionBy('steamid', 'notoriety')
n_revs_each_notoriety = base_reviews_and_games.to_spark().withColumn('n_revs_each_notoriety', F.count(col('notoriety')).over(w1))
n_revs_each_notoriety = n_revs_each_notoriety.dropDuplicates(['steamid', 'notoriety'])

# Obtain for each user the maximum number of reviews he/she wrote for each level of notoriety
w2 = Window.partitionBy('steamid')
max_revs_each_notoriety = n_revs_each_notoriety.withColumn('max_revs_each_notoriety', F.max(col('n_revs_each_notoriety')).over(w2))
max_revs_each_notoriety = max_revs_each_notoriety.dropDuplicates(['steamid'])

# Obtain for each user the overall number of written reviews
n_revs = n_revs_each_notoriety.withColumn('n_revs', F.sum(col('n_revs_each_notoriety')).over(w2))
n_revs = n_revs.dropDuplicates(['steamid'])

# Select the users that have written at least three reviews
at_least_three_games = n_revs[n_revs['n_revs'] > 2]

# Obtain for each user the max loyalty to one notoriety level (no matter which one)
users_max_loyalty_perc = max_revs_each_notoriety.join(at_least_three_games[['steamid', 'n_revs']], on='steamid')\
                                                .withColumn('max_user_loyalty', 
                                                            col('max_revs_each_notoriety') / col('n_revs') * 100)\
                                                .to_pandas_on_spark()\
                                                [['steamid','max_user_loyalty']]

users_max_loyalty_perc.to_spark().limit(100).toPandas()

,steamid,max_user_loyalty
0,76561197960267615,50.000000
1,76561197960268417,100.000000
2,76561197960269230,100.000000
3,76561197960269409,62.500000
4,76561197960269579,100.000000
...,...,...
95,76561197960414332,100.000000
96,76561197960416558,50.000000
97,76561197960416994,100.000000
98,76561197960419349,66.666667


### Analysis
<br>

#### Statistical test

Are users loyal, for a M%, to one notoriety level (no matter which one)?

In [8]:
# Perform the mean test
statistics, p_value = utils.mean_test(users_max_loyalty_perc['max_user_loyalty'], 85)

print(f"Mean test coefficient: {statistics:.3f} \nP-value: {p_value:.3f}")
alpha = 0.05
if p_value < alpha:
    print('\nReject the null hypothesis')
else:
    print('\nDo not reject the null hypothesis')

Mean test coefficient: 42.216 
P-value: 0.000

Reject the null hypothesis


We can notice that our hypothesis is true (and is statistically significant).<br> 
So we can say that people are loyal to review (and play) the 85% of same notoriety level games. Furthermore, considering a larger population the percentage increases.

## 7) Do players who mostly review famous games also enjoy niche ones?

#### Obtain the loyalty level for famous & niche games, of players that have at least reviewed 3 games

In [9]:
# Obtain for each user the percentage of voted up for each notoriety level
perc_voted_up_each_notoriety = n_revs_each_notoriety.withColumn('perc_voted_up_each_notoriety',
                                                                F.mean(col('voted_up').cast('int') * 100).over(w1))
perc_voted_up_each_notoriety = perc_voted_up_each_notoriety.dropDuplicates(['steamid', 'notoriety'])

# Select the users that have written at least three reviews
at_least_three_games = perc_voted_up_each_notoriety.join(at_least_three_games[['steamid', 'n_revs']], on='steamid')

# Obtain for each user the loyalty of each notoriety level
loyalty_each_level_notoriety = at_least_three_games.withColumn('loyalty', col('n_revs_each_notoriety') / col('n_revs') * 100)

#### Consider only people who are very loyal to famous games and that have at least played one niche game

In [10]:
df = loyalty_each_level_notoriety.to_pandas_on_spark()

# Obtain people that have reviewed at least one niche game
loyalty_niche_not_zero = df[(df['notoriety'] == 'NICHE') & (df['loyalty'] > 0.0)]
loyalty_niche_not_zero = loyalty_niche_not_zero.rename(columns={'loyalty': 'niche_loyalty', 'perc_voted_up_each_notoriety': 'perc_niche_liked'})

# Obtain people that have reviewed for the majority of cases famous games (higher than 55%)
loyalty_famous_high = df[(df['notoriety'] == 'FAMOUS') & (df['loyalty'] > 55.0)]
loyalty_famous_high = loyalty_famous_high.rename(columns={'loyalty': 'famous_loyalty'})

# Merge the two obtained datasets
loyalties_df = loyalty_niche_not_zero.merge(loyalty_famous_high[['steamid', 'famous_loyalty']])[['steamid', 'n_revs', 'famous_loyalty', 'niche_loyalty', 'perc_niche_liked']]
loyalties_df.to_spark().limit(100).toPandas()

,steamid,n_revs,famous_loyalty,niche_loyalty,perc_niche_liked
0,76561197960319712,3,66.666667,33.333333,0.0
1,76561197960427320,11,72.727273,9.090909,100.0
2,76561197960478145,26,65.384615,3.846154,100.0
3,76561197960967170,3,66.666667,33.333333,0.0
4,76561197961194018,18,55.555556,5.555556,0.0
...,...,...,...,...,...
95,76561197993727469,3,66.666667,33.333333,100.0
96,76561197993729693,4,75.000000,25.000000,100.0
97,76561197993837300,9,66.666667,11.111111,100.0
98,76561197994208160,3,66.666667,33.333333,0.0


### Analysis
<br>

#### Statistical test

Are users loyal to famous notoriety games also enjoying M% of niche ones?

In [11]:
statistics, p_value = utils.mean_test(loyalties_df['perc_niche_liked'], 60)

print(f"Mean test coefficient: {statistics:.3f} \nP-value: {p_value:.3f}")
alpha = 0.05
if p_value < alpha:
    print('\nReject the null hypothesis')
else:
    print('\nDo not reject the null hypothesis')

Mean test coefficient: 2.907 
P-value: 0.002

Reject the null hypothesis


We can notice that our hypothesis is true (and is statistically significant).<br> 
So we can say that players loyal to famous notoriety games also enjoyed 60% of niche ones. In this case, considering a larger population does not change the obtained results.

In [12]:
# Stop the Spark context underlying the Spark session
spark.stop()